In [0]:
#Questions
# Who are the winners of the D1 division in the Germany Football Association (Bundesliga) in the last decade?
# Which teams have been relegated in the past 10 years?
# Does Octoberfest affect the performance of Bundesliga?
# Which season of Bundesliga was the most competitive in the last decade?
# What's the best month to watch Bundesliga?

#spark = SparkSession \
#        .builder \
#        .appName("firstSpark") \
#       .getOrCreate()

from pyspark import SparkContext
from pyspark.sql import Window, Row
from pyspark.sql.functions import *
from pyspark.sql.types import *
import matplotlib.pyplot as plt

#reading file
file = '/FileStore/tables/Matches.csv'
df = spark \
  .read.format("csv") \
  .option("inferSchema", "True") \
  .option("header", "True") \
  .csv(file)
display(df.limit(15))



Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A
6,D2,2009,2010-04-03,Union Berlin,Karlsruhe,1,1,D
7,D2,2009,2009-08-14,Paderborn,Karlsruhe,2,0,H
8,D2,2009,2010-03-08,Bielefeld,Karlsruhe,0,1,A
9,D2,2009,2009-09-26,Kaiserslautern,Karlsruhe,2,0,H
10,D2,2009,2009-11-21,Hansa Rostock,Karlsruhe,2,1,H


In [0]:
df = df.withColumnRenamed('FTHG', 'HomeTeamGoals')
df = df.withColumnRenamed('FTAG', 'AwayTeamGoals')
df = df.withColumnRenamed('FTR', 'FinalResult')


In [0]:
display(df.limit(20))

Match_ID,Div,Season,Date,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,FinalResult
1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A
6,D2,2009,2010-04-03,Union Berlin,Karlsruhe,1,1,D
7,D2,2009,2009-08-14,Paderborn,Karlsruhe,2,0,H
8,D2,2009,2010-03-08,Bielefeld,Karlsruhe,0,1,A
9,D2,2009,2009-09-26,Kaiserslautern,Karlsruhe,2,0,H
10,D2,2009,2009-11-21,Hansa Rostock,Karlsruhe,2,1,H


In [0]:
# Create 3 column with game results
df = df.withColumn('HomeTeamWin', when(df['FinalResult'] == 'H', 1).otherwise(0)) \
        .withColumn('AwayTeamWin', when(df['FinalResult'] == 'A', 1).otherwise(0)) \
        .withColumn('GameTie', when(df['FinalResult'] == 'D', 1).otherwise(0)) \

display(df.limit(20))

Match_ID,Div,Season,Date,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,FinalResult,HomeTeamWin,AwayTeamWin,GameTie
1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H,1,0,0
2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A,0,1,0
3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D,0,0,1
4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H,1,0,0
5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A,0,1,0
6,D2,2009,2010-04-03,Union Berlin,Karlsruhe,1,1,D,0,0,1
7,D2,2009,2009-08-14,Paderborn,Karlsruhe,2,0,H,1,0,0
8,D2,2009,2010-03-08,Bielefeld,Karlsruhe,0,1,A,0,1,0
9,D2,2009,2009-09-26,Kaiserslautern,Karlsruhe,2,0,H,1,0,0
10,D2,2009,2009-11-21,Hansa Rostock,Karlsruhe,2,1,H,1,0,0


In [0]:
# now we get the 2000-2010 period from busdesliga
bundesliga = df.filter( (df.Season >= 2000) & (df.Season  <= 2010) & (df.Div == 'D1'))
display(bundesliga.limit(40))

Match_ID,Div,Season,Date,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,FinalResult,HomeTeamWin,AwayTeamWin,GameTie
21,D1,2009,2010-02-06,Bochum,Leverkusen,1,1,D,0,0,1
22,D1,2009,2009-11-22,Bayern Munich,Leverkusen,1,1,D,0,0,1
23,D1,2009,2010-05-08,M'gladbach,Leverkusen,1,1,D,0,0,1
24,D1,2009,2009-08-08,Mainz,Leverkusen,2,2,D,0,0,1
25,D1,2009,2009-10-17,Hamburg,Leverkusen,0,0,D,0,0,1
26,D1,2009,2010-04-17,Stuttgart,Leverkusen,2,1,H,1,0,0
27,D1,2009,2010-03-20,Dortmund,Leverkusen,3,0,H,1,0,0
28,D1,2009,2009-10-31,Schalke 04,Leverkusen,2,2,D,0,0,1
29,D1,2009,2009-08-22,Freiburg,Leverkusen,0,5,A,0,1,0
30,D1,2009,2010-02-21,Werder Bremen,Leverkusen,2,2,D,0,0,1


In [0]:
#home team features
home = bundesliga.groupBy('Season', 'HomeTeam') \
  .agg(sum('HomeTeamWin').alias('TotalHomeWin'),
      sum('AwayTeamWin').alias('TotalHomeLoss'),
      sum('GameTie').alias('TotalHomeTie'),
      sum('HomeTeamGoals').alias('HomeScoredGoals'),
      sum('AwayTeamGoals').alias('HomeAgainstGoals')) \
    .withColumnRenamed('HomeTeam', 'Team')

#away team features
away = bundesliga.groupBy('Season', 'AwayTeam') \
  .agg(sum('AwayTeamWin').alias('TotalAwayWin'),
      sum('HomeTeamWin').alias('TotalAwayLoss'),
      sum('GameTie').alias('TotalAwayTie'),
      sum('AwayTeamGoals').alias('AwayScoredGoals'),
      sum('AwayTeamGoals').alias('AwayAgainstGoals')) \
    .withColumnRenamed('AwayTeam', 'Team')

display(home.limit(10))
display(away.limit(10))

Season,Team,TotalHomeWin,TotalHomeLoss,TotalHomeTie,HomeScoredGoals,HomeAgainstGoals
2005,Bayern Munich,14,1,2,42,14
2008,M'gladbach,5,8,4,23,27
2006,Cottbus,6,6,5,21,22
2010,Hannover,12,4,1,32,17
2004,Leverkusen,12,2,3,42,18
2007,Cottbus,8,7,2,25,20
2004,Mainz,9,5,3,28,21
2004,Hannover,8,7,2,21,19
2003,FC Koln,6,9,2,22,23
2001,Hansa Rostock,6,5,6,20,18


Season,Team,TotalAwayWin,TotalAwayLoss,TotalAwayTie,AwayScoredGoals,AwayAgainstGoals
2005,Bayern Munich,8,2,7,25,25
2008,M'gladbach,3,11,3,16,16
2006,Cottbus,5,9,3,17,17
2010,Hannover,7,8,2,17,17
2004,Leverkusen,4,7,6,23,23
2007,Cottbus,1,9,7,10,10
2004,Hannover,5,8,4,13,13
2004,Mainz,3,10,4,22,22
2003,FC Koln,0,14,3,10,10
2001,Hansa Rostock,3,13,1,15,15


In [0]:
#season features
window = ['Season']
window = Window.partitionBy(window).orderBy(col('Pts').desc(), col('GoalDifferentials').desc())
table = home.join(away, ['Team', 'Season'],  'inner') \
    .withColumn('GoalsScored', col('HomeScoredGoals') + col('AwayScoredGoals')) \
    .withColumn('GoalsAgainst', col('HomeAgainstGoals') + col('AwayAgainstGoals')) \
    .withColumn('GoalDifferentials', col('GoalsScored') - col('GoalsAgainst')) \
    .withColumn('Win', col('TotalHomeWin') + col('TotalAwayWin')) \
    .withColumn('Loss', col('TotalHomeLoss') + col('TotalAwayLoss')) \
    .withColumn('Tie', col('TotalHomeTie') + col('TotalAwayTie')) \
    .withColumn('Matches', col('Win') + col('Loss') + col('Tie')) \
    .withColumn('Pts', (col('Win')*3) + (col('Tie'))) \
    .withColumn('WinPct', round((100* col('Win')/(col('Win') + col('Loss') + col('Tie'))), 2)) \
    .drop('HomeScoredGoals', 'AwayScoredGoals', 'HomeAgainstGoals', 'AwayAgainstGoals') \
    .drop('TotalHomeWin', 'TotalAwayWin', 'TotalHomeLoss', 'TotalAwayLoss', 'TotalHomeTie', 'TotalAwayTie') \
    .withColumn('TeamPosition', rank().over(window)) 

#table_df = table.filter(col('TeamPosition') == 1).orderBy(asc('Season'))
table = table.select("Team", "Season", "Matches", "Pts", "Win", "Tie", "Loss", "GoalsScored", 
                     "GoalsAgainst", "GoalDifferentials", "WinPct", "TeamPosition")
display(table.limit(20))

Team,Season,Matches,Pts,Win,Tie,Loss,GoalsScored,GoalsAgainst,GoalDifferentials,WinPct,TeamPosition
Bayern Munich,2000,34,63,19,6,9,62,45,17,55.88,1
Schalke 04,2000,34,62,18,8,8,65,42,23,52.94,2
Dortmund,2000,34,58,16,10,8,62,48,14,47.06,3
Leverkusen,2000,34,57,17,6,11,54,40,14,50.0,4
Hertha,2000,34,56,18,2,14,58,42,16,52.94,5
Freiburg,2000,34,55,15,10,9,54,33,21,44.12,6
Werder Bremen,2000,34,53,15,8,11,53,38,15,44.12,7
Kaiserslautern,2000,34,50,15,5,14,49,44,5,44.12,8
Wolfsburg,2000,34,47,12,11,11,60,40,20,35.29,9
FC Koln,2000,34,46,12,10,12,59,44,15,35.29,10


In [0]:
#relegated teams bundesliga 2000-2010
relegated = table.filter((table.TeamPosition > 15))
display(relegated)

Team,Season,Matches,Pts,Win,Tie,Loss,GoalsScored,GoalsAgainst,GoalDifferentials,WinPct,TeamPosition
Ein Frankfurt,2000,34,35,10,5,19,41,37,4,29.41,16
Unterhaching,2000,34,35,8,11,15,35,34,1,23.53,17
Bochum,2000,34,27,7,6,21,30,38,-8,20.59,18
Freiburg,2001,34,30,7,9,18,37,38,-1,20.59,16
FC Koln,2001,34,29,7,8,19,26,31,-5,20.59,17
St Pauli,2001,34,22,4,10,20,37,46,-9,11.76,18
Bielefeld,2002,34,36,8,12,14,35,28,7,23.53,16
Nurnberg,2002,34,30,8,6,20,33,41,-8,23.53,17
Cottbus,2002,34,30,7,9,18,34,47,-13,20.59,18
Ein Frankfurt,2003,34,32,9,5,20,36,35,1,26.47,16
